# IT Ticket Classifier — DHAUZ Challenge

Notebook executável: carrega o dataset, amostra 200 tickets, monta o RAG (embeddings + FAISS), executa o fluxo LangGraph em exemplos e na amostra, calcula métricas.
Arquitetura baseada em pesquisa com experimentação em "labs/architecture-comparison.ipynb"

### Resultados do labs (architecture-comparison)

Gráfico de threshold KNN (cruzamento coverage × F1 macro) e tabela comparativa de modelos no conjunto de teste (hold-out excl. sample_200).

![KNN: métricas vs confidence threshold](charts/threshold.png)
Resultado mostra que threshold para maior coverage e maio f1 macro é 0.45
Cruzamento (coverage ≈ F1 macro): threshold=0.45 → coverage=90.35%, F1 macro=0.8697, accuracy=0.8680
Melhor equilíbrio (max coverage×F1): threshold=0.35 → coverage=95.31%, F1 macro=0.8450, accuracy=0.8424

**Tabela comparativa (test set):**

| model    | accuracy | f1_macro | f1_weighted |
|----------|----------|----------|-------------|
| RNN      | 0.772571 | 0.763023 | 0.773050    |
| LSTM     | 0.800138 | 0.794236 | 0.801268    |
| BiLSTM   | 0.831840 | 0.835082 | 0.832611    |
| BiGRU    | 0.833218 | 0.838499 | 0.833300    |
| CNN+BiGRU| 0.837354 | 0.842469 | 0.837848    |
| KNN      | 0.833218 | 0.832331 | 0.832036    |

### usaremos KNN pelo motivo de ser mais explicável e permitir construir uma base de conhecimento muito granular, veja mais em labs/architecture-comparison.ipynb

In [1]:
import os
import sys
import numpy as np
from pathlib import Path

ROOT = Path(".").resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from dotenv import load_dotenv
load_dotenv(ROOT / ".env")

import config
np.random.seed(config.SEED)

## 0. Baixar dataset do Kaggle (só se ainda não tiver o CSV em data/raw)

In [2]:
from src.prep import download_from_kaggle

path = download_from_kaggle()
print(f"Dataset em: {path}")

Dataset em: /Users/moises/Documents/ticket-classifier/data/raw/all_tickets_processed_improved_v3.csv


## 1. Dataset completo → vector store; sample_200 → test set

In [3]:
import pandas as pd
from src.prep import document_text, load_dataset, get_text_and_label_columns, stratified_sample
from src.rag import VectorStore

df_full = load_dataset()
text_cols, label_col = get_text_and_label_columns(df_full)
classes = sorted(set(df_full[label_col].astype(str)))

n_sample = min(config.SAMPLE_SIZE, len(df_full))
df_sample = stratified_sample(df_full, label_col, n=n_sample)
df_sample.to_csv(config.DATA_PROCESSED / "sample_200.csv", index=False)
ids_test = set(df_sample["id"])
df_train = df_full[~df_full["id"].isin(ids_test)]

texts_train = [document_text(row, text_cols) for _, row in df_train.iterrows()]
labels_train = df_train[label_col].astype(str).tolist()
ids_train = df_train["id"].tolist()

artifact_path = config.ARTIFACTS_DIR
if (artifact_path / "index.faiss").exists():
    store = VectorStore.load(artifact_path)
    print("Vector store carregado de:", artifact_path)
else:
    vc = VectorStore()
    store = vc.build(texts_train, labels_train, ids=ids_train)
    store.save(artifact_path)
    print("Vector store construído (apenas train) e salvo em:", artifact_path)

texts = [document_text(row, text_cols) for _, row in df_sample.iterrows()]

print("Classes:", classes)
print("Vector store (train): %d documentos" % len(df_train))
print("Test set (sample_200): %d tickets" % len(df_sample))

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Vector store carregado de: /Users/moises/Documents/ticket-classifier/outputs/artifacts
Classes: ['Access', 'Administrative rights', 'HR Support', 'Hardware', 'Internal Project', 'Miscellaneous', 'Purchase', 'Storage']
Vector store (train): 47637 documentos
Test set (sample_200): 200 tickets


In [4]:
print("Distribuição por classe:")
print(df_sample[label_col].value_counts())

Distribuição por classe:
Topic_group
Hardware                 25
Access                   25
Miscellaneous            25
HR Support               25
Purchase                 25
Administrative rights    25
Storage                  25
Internal Project         25
Name: count, dtype: int64


## 2. Inferência em exemplos


In [5]:
# store, classes e texts já carregados na célula 1.
import logging
from tqdm.auto import tqdm
from src.graph import build_pipeline, run_pipeline

logging.getLogger("ticket_classifier.llm_usage").setLevel(logging.WARNING)

compiled, _, _ = build_pipeline(store, classes)

for i in tqdm(range(min(3, len(texts))), desc="Exemplos"):
    out = run_pipeline(compiled, texts[i], classes)
    print(f"--- Ticket {i+1} ---")
    print("Texto (trecho):", texts[i][:150], "...")
    print("Saída:", out)
    print()

Exemplos:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Ticket:
monitor request vulcan friday october pm hello please log each user monitor allocation user vulcan thank weekend engineer friday october vulcan parte ne va respective va

Classe atribuída: Hardware. Confiança KNN: 1.00.

Vizinhos KNN que votaram nessa classe:
- #0 (dist 0.59): monitor for vulcan thursday october monitor vulcan hi want let vulcan more working monitor assigned vulcan monitor out r...
- #1 (dist 0.63): monitor en november pm monitor hello please log retrieve monitor thank engineer
- #2 (dist 0.67): temporary monitor request friday monitor hello please log assigning monitor he needs monitor out his tasks thank enginee...
- #3 (dist 0.68): monitor request friday pm monitor dear please provide monitor thank senior developer
- #4 (dist 0.68): new delivery tuesday july pm hello received monitor po please advise please log thank engineer
- #5 (dist 0.69): monitor allocation july pm monitor allocation dear please log assign allocating monitor thanks kind regards engineer

llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64

--- Ticket 1 ---
Texto (trecho): monitor request vulcan friday october pm hello please log each user monitor allocation user vulcan thank weekend engineer friday october vulcan parte  ...
Saída: {'classe': 'Hardware', 'justificativa': "O termo 'monitor' é frequentemente mencionado no ticket, o que está correlacionado com a classe atribuída 'Hardware'. Além disso, os vizinhos KNN também mencionam 'monitor' em seus textos, como no #0 'monitor for vulcan' e no #1 'monitor hello please log retrieve monitor'. Outro termo que está correlacionado é 'allocation', presente no ticket e nos vizinhos #5 e #6, que também mencionam 'allocation'.", 'classification_source': 'knn', 'confidence': 1.0, 'inference_time_sec': 42.93831883301027, 'classification_tokens': None, 'justification_tokens': 536}

Ticket:
stopped when docker start was executed sent wednesday february hi we having same problem we had few days ago server was stopped when executed docker start looks like if machine have enough resource

## 3. Rodar na amostra de 200 e salvar resultados

In [6]:
from src.logging_utils import log_result
from tqdm.auto import tqdm

results_path = config.OUTPUTS / "results_sample.jsonl"
if results_path.exists():
    results_path.unlink()

predictions = []
for pos, (_, row) in enumerate(tqdm(list(df_sample.iterrows()), desc="Pipeline")):
    text = document_text(row, text_cols)
    out = run_pipeline(compiled, text, classes, thread_id=str(pos), instance_id=row["id"])
    pred = out["classe"]
    predictions.append(pred)
    log_result({
        "id": row["id"],
        "ticket_index": pos,
        "true": row[label_col],
        "pred": pred,
        "justificativa": out["justificativa"],
        "classification_source": out.get("classification_source"),
        "confidence": out.get("confidence"),
        "inference_time_sec": out.get("inference_time_sec"),
        "classification_tokens": out.get("classification_tokens"),
        "justification_tokens": out.get("justification_tokens"),
    })

print(f"Salvos {len(predictions)} resultados em {results_path}")

Pipeline:   0%|          | 0/200 [00:00<?, ?it/s]

Ticket:
monitor request vulcan friday october pm hello please log each user monitor allocation user vulcan thank weekend engineer friday october vulcan parte ne va respective va

Classe atribuída: Hardware. Confiança KNN: 1.00.

Vizinhos KNN que votaram nessa classe:
- #0 (dist 0.59): monitor for vulcan thursday october monitor vulcan hi want let vulcan more working monitor assigned vulcan monitor out r...
- #1 (dist 0.63): monitor en november pm monitor hello please log retrieve monitor thank engineer
- #2 (dist 0.67): temporary monitor request friday monitor hello please log assigning monitor he needs monitor out his tasks thank enginee...
- #3 (dist 0.68): monitor request friday pm monitor dear please provide monitor thank senior developer
- #4 (dist 0.68): new delivery tuesday july pm hello received monitor po please advise please log thank engineer
- #5 (dist 0.69): monitor allocation july pm monitor allocation dear please log assign allocating monitor thanks kind regards engineer

KeyboardInterrupt: 

## 4. Métricas e relatório

In [ ]:
from src.metrics import compute_metrics, save_metrics_report

y_true = df_sample[label_col].astype(str).tolist()
metrics = compute_metrics(y_true, predictions, labels=classes)
save_metrics_report(metrics)

print("Accuracy:", metrics["accuracy"])
print("F1 macro:", metrics["f1_macro"])
print("F1 weighted:", metrics["f1_weighted"])
print("\nClassification report:")
for k, v in metrics["classification_report"].items():
    if isinstance(v, dict):
        print(k, v)
    else:
        print(k, v)

## 5. Exemplo de saída JSON

In [ ]:
import json
ex = run_pipeline(compiled, texts[0], classes)
print(json.dumps(ex, indent=2, ensure_ascii=False))

---

In [ ]:
import json
ex = run_pipeline(compiled, texts[0], classes)
print(json.dumps(ex, indent=2, ensure_ascii=False))